# 0.0 Imports

In [ ]:
pip install seaborn

In [1]:
import math
import datetime
import inflection

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import Image

## 0.1 Helper Functions

## 0.2 Loading Data

In [ ]:
df_sales_raw = pd.read_csv( 'datasets/train.csv', low_memory=False )
df_store_raw = pd.read_csv( 'datasets/store.csv', low_memory=False )
                           
#merge
df_raw = pd.merge( df_sales_raw, df_store_raw, how='left', on='Store')

# 1.0 Descrição dos Dados

In [ ]:
df1 = df_raw.copy()

## 1.1 Rename Columns

In [ ]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 
            'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 
            'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']

snakecase = lambda x: inflection.underscore( x )

cols_new = list(map( snakecase, cols_old))

# rename
df1.columns = cols_new

## 1.2 Data Dimension

In [ ]:
print(f'Number of Rows{df1.shape[0]}')
print(f'Number of Cols{df1.shape[1]}')

## 1.3 Data Types

In [ ]:
df1['date'] = pd.to_datetime( df1['date'])
df1.dtypes

## 1.4 Check NA

In [ ]:
df1.isna().sum()

## 1.5 Fillout NA

In [ ]:
df1['competition_distance'].max()

In [ ]:
# competition_distance    
df1['competition_distance'] = df1['competition_distance'].apply( lambda x: 200000.0 if math.isnan( x ) else x)

# competition_open_since_month
df1['competition_open_since_month'] = df1.apply( lambda x: x['date'].month if math.isnan( x['competition_open_since_month'] ) else x['competition_open_since_month'], axis=1)

# competition_open_since_year    
df1['competition_open_since_year'] = df1.apply( lambda x: x['date'].year if math.isnan( x['competition_open_since_year'] ) else x['competition_open_since_year'], axis=1)

# promo2_since_week              
df1['promo2_since_week'] = df1.apply( lambda x: x['date'].week if math.isnan( x['promo2_since_week'] ) else x['promo2_since_week'], axis=1)

# promo2_since_year              
df1['promo2_since_year'] = df1.apply( lambda x: x['date'].year if math.isnan( x['promo2_since_year'] ) else x['promo2_since_year'], axis=1)

# promo_interval 
month_map = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec' }
df1['promo_interval'].fillna(0, inplace=True)
df1['month_map'] = df1['date'].dt.month.map( month_map )
df1['is_promo'] = df1[['promo_interval', 'month_map']].apply( lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split( ',' ) else 0, axis=1 )

df1.isna().sum()

## 1.6 Change Types

In [ ]:
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype( int )
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype( int )

df1['promo2_since_week'] = df1['promo2_since_week'].astype( int )
df1['promo2_since_year'] = df1['promo2_since_year'].astype( int )

df1.dtypes

In [ ]:
#df1.to_csv('df1.csv')

In [ ]:
#df2 = pd.read_csv( 'df1.csv', low_memory=False )

## 1.7 Descriptive Statistical

In [ ]:
num_attributes = df1.select_dtypes( include = ['int64', 'float64'])
cat_attributes = df1.select_dtypes( exclude = ['int64', 'float64', 'datetime64[ns]'])

### 1.7.1 Numerical Attributes

In [ ]:
# Central tendency - mean, median
ct1 = pd.DataFrame( num_attributes.apply( np.mean )).T
ct2 = pd.DataFrame( num_attributes.apply( np.median )).T

# dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame( num_attributes.apply( np.std )).T
d2 = pd.DataFrame( num_attributes.apply( min )).T
d3 = pd.DataFrame( num_attributes.apply( max )).T
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() )).T
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() )).T
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() )).T

#concat
m = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6] ).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis'] 

In [ ]:
m

In [ ]:
plt.figure(figsize=(18, 10))
sns.distplot( df1['competition_distance'] )

### 1.7.2 Categorical Attributes

In [ ]:
cat_attributes.apply( lambda x: x.unique().shape[0])

In [ ]:
plt.figure(figsize=(18, 10))

aux1 = df1[(df1['state_holiday'] !='0') & (df1['sales']>0)]

plt.subplot( 1, 3, 1)
sns.boxplot( x= 'state_holiday', y= 'sales', data=aux1)

plt.subplot( 1, 3, 2)
sns.boxplot( x= 'store_type', y= 'sales', data=aux1)

plt.subplot( 1, 3, 3)
sns.boxplot( x= 'assortment', y= 'sales', data=aux1)

# 2.0 Feature Engineering

In [ ]:
df2 = df1.copy()

## 2.1 Mapa Mental de Hipóteses

In [ ]:
Image( 'img/MindMapHypothesis.png' )

## 2.2 Criação da Hipóteses

### 2.2.1 Hipóteses Loja

    -1 Lojas com maior quadro de funcionários deveriam vender mais
    
    -2 Lojas com maior estoque deveriam vender mais
    
    -3 Lojas com maior porte deveriam vender mais
    
    -4 Lojas com maior sortimento deveriam vender mais
    
    -5 Lojas com competidores mais próximos deveriam vender menos
    
    -6 Lojas com competidores à mais tempo deveriam vender mais

### 2.2.1 Hipóteses Produto

    -1 Lojas que investem mais em marketing deveriam vender mais
    
    -2 Lojas que expoe mais os produtos nas vitrines deveriam vender mais
    
    -3 Lojas que tem preços menores nos produtos deveriam vender mais
    
    -4 Lojas que possuem preços menores por mais tempo deveriam vender mais 
    
    -5 Lojas com descontos maiores deveriam vender mais
    
    -6 Lojas com promoções ativas por mais tempo deveriam vender mais
    
    -7 Lojas com mais dias de promoção deveriam vender mais
    
    -8 Lojas com mais promoções consecutivas deveriam vender mais

### 2.2.1 Hipóteses Tempo

    -1 Lojas abertas durante o feriado de natal deveriam vender mais
    
    -2 Lojas deveriam vender mais ao longo dos anos
    
    -3 Lojas deveriam vender mais no segundo semestre do ano
    
    -4 Lojas deveriam vender mais depois do dia 10 do mês
    
    -5 Lojas deveriam vender menos aos finais de semana
    
    -6 Lojas deveriam vender menos durante os feriados escolares

## 2.3 Lista Final de Hipótesis

    -1 Lojas com maior sortimento deveriam vender mais
     
    -2 Lojas com competidores mais próximos deveriam vender menos
    
    -3 Lojas com competidores à mais tempo deveriam vender mais
     
    -4 Lojas com promoções ativas por mais tempo deveriam vender mais

    -5 Lojas com mais dias de promoção deveriam vender mais

    -6 Lojas com mais promoções consecutivas deveriam vender mais
      
    -7 Lojas abertas durante o feriado de natal deveriam vender mais
   
    -8 Lojas deveriam vender mais ao longo dos anos
    
    -9 Lojas deveriam vender mais no segundo semestre do ano
    
    -10 Lojas deveriam vender mais depois do dia 10 do mês
    
    -11 Lojas deveriam vender menos aos finais de semana
    
    -12 Lojas deveriam vender menos durante os feriados escolares    

## 2.2 Criação da Hipóteses

In [ ]:
df2.head().T

In [ ]:
# year
df2['year'] = df2['date'].dt.year

#month
df2['month'] = df2['date'].dt.month

#day
df2['day'] = df2['date'].dt.day

#week of year
df2['week_of_year'] = df2['date'].dt.weekofyear

#year week
df2['year_week'] = df2['date'].dt.strftime( '%Y-%W' )

#competition since
df2['competition_since'] = df2.apply(lambda x: datetime.datetime( year=x['competition_open_since_year'], month=x['competition_open_since_month'], day=1), axis=1)
df2['competition_since_month'] = ((df2['date'] - df2['competition_since'])/30).apply(lambda x: x.days).astype(int)

#promo since
df2['promo_since'] = df2['promo2_since_year'].astype(str) + '-' + df2['promo2_since_week'].astype(str)
df2['promo_since'] = df2['promo_since'].apply(lambda x: datetime.datetime.strptime( x + '-1', '%Y-%W-%w' ) - datetime.timedelta( days=7 ))
df2['promo_time_week'] = ( ( df2['date'] - df2['promo_since'] )/7 ).apply(lambda x: x.days ).astype( int )

#assortment / a=basic, b=extra, c=extended
df2['assortment'] = df2['assortment'].apply(lambda x: 'basic' if x == 'a' else 'extra' if x == 'b' else 'extended')

#state holiday / a=public holiday, b=easter holiday, c=christmas
df2['state_holiday'] = df2['state_holiday'].apply(lambda x: 'public_holiday' if x == 'a' else 'easter_holiday' if x == 'b' else 'christmas' if x =='c' else 'regular_day')


In [ ]:
df2.head()

In [ ]:
df2.to_csv('df2.csv', index_label=False)

# 3.0 Filtragem de Variáveis

In [3]:
df3 = pd.read_csv( 'df2.csv', low_memory=False )

In [4]:
df3.head()

,store,day_of_week,date,sales,customers,open,promo,state_holiday,school_holiday,store_type,...,is_promo,year,month,day,week_of_year,year_week,competition_since,competition_since_month,promo_since,promo_time_week
0,1,5,2015-07-31,5263,555,1,1,regular_day,1,c,...,0,2015,7,31,31,2015-30,2008-09-01,84,2015-07-27,0
1,2,5,2015-07-31,6064,625,1,1,regular_day,1,a,...,1,2015,7,31,31,2015-30,2007-11-01,94,2010-03-22,279
2,3,5,2015-07-31,8314,821,1,1,regular_day,1,a,...,1,2015,7,31,31,2015-30,2006-12-01,105,2011-03-28,226
3,4,5,2015-07-31,13995,1498,1,1,regular_day,1,c,...,0,2015,7,31,31,2015-30,2009-09-01,71,2015-07-27,0
4,5,5,2015-07-31,4822,559,1,1,regular_day,1,a,...,0,2015,7,31,31,2015-30,2015-04-01,4,2015-07-27,0


## 3.1 Filtragem das Linhas

In [5]:
df3 = df3[(df3['open'] != 0) & (df3['sales'] > 0)]

## 3.2 Seleção das Colunas

In [9]:
#cols_drop = ['customers', 'open', 'promo_interval', 'month_map']
df3 = df3.drop( columns = ['customers', 'open', 'promo_interval', 'month_map'] )

In [11]:
df3.columns

Index(['store', 'day_of_week', 'date', 'sales', 'promo', 'state_holiday',
       'school_holiday', 'store_type', 'assortment', 'competition_distance',
       'competition_open_since_month', 'competition_open_since_year', 'promo2',
       'promo2_since_week', 'promo2_since_year', 'is_promo', 'year', 'month',
       'day', 'week_of_year', 'year_week', 'competition_since',
       'competition_since_month', 'promo_since', 'promo_time_week'],
      dtype='object')